# Creating a Mock Galaxy Catalog With Alignment

Here we show how to create an HOD model that has alignment information using halotools built-in catalogs

In [1]:
# The imports shown here are the most basic needed
# Extra imports for using the SubhaloAlignment component and for using IA strength models are given below in their respective subsections
import numpy as np
from halotools.empirical_models import HodModelFactory
from halotools.empirical_models import TrivialPhaseSpace, SubhaloPhaseSpace, Zheng07Cens, Zheng07Sats
from halotools.empirical_models.ia_models.ia_model_components import CentralAlignment, RadialSatelliteAlignment, SubhaloAlignment

from halotools.sim_manager import CachedHaloCatalog

The following masking function is just my personal way of ignoring halos without axis values

In [2]:
# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    halocat._halo_table = halocat.halo_table[ ~bad_mask ]

First, read in the cached halo catalog (below).</br>
IMPORTANT: Run this line only once during each kernel session (restarting the kernel and running again is fine).</br>
If you try to rerun this line with an existing, you will get errors.

In [3]:
halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
mask_bad_halocat(halocat)

## Creating a basic (no IA strength model) HOD model with galaxy alignment

Instantiate the separate model components</br>
Normally, halotools will take arguments for occupation and profile models for central and satellite galaxy populations.</br>
We have added the ability to pass in alignment model components for each as well

### Using a radial satellite alignment model
There are different alignment models for the centrals and satellites. Here, we look at the RadialSatellitesAlignment model for the satellite galaxies.</br>
This is where the satellite galaxy will align with respect to the radial vector between it and its central galaxy. This model uses the default</br>
parameter dictionary (explained in subhalo alignment model section), meaning we don't have to think about it yet.

In [4]:
# MODELS

# Alignment Strengths
satellite_alignment = 0.6
central_alignment = 0.8

# Central galaxy components
cens_occ_model = Zheng07Cens()
cens_prof_model = TrivialPhaseSpace()
cens_orientation_model = CentralAlignment(central_alignment_strength=central_alignment)

# Satellite galaxy Components
sats_occ_model = Zheng07Sats()
prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
sats_prof_model = SubhaloPhaseSpace(*prof_args)
sats_orientation_model = RadialSatelliteAlignment(satellite_alignment_strength=satellite_alignment)

When building an HOD model, we pass in these different model components. The order in which we give these is very important.</br>
Occupation models must come before the profile, since the number of galaxies must be determined before we can place them. Similarly,</br>
the profile must come before the orientation.

In [5]:
# Initially create the mock with SubhaloAlignment to overwrite the host information with subhalo
model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     centrals_orientation = cens_orientation_model,
                                     satellites_orientation = sats_orientation_model,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )

#seed=132358712
seed=None
model_instance.populate_mock(halocat, seed=seed)

/media/nvanalfen/T7/Northeastern/Research/my halotools/halotools/halotools/empirical_models/ia_models/ia_model_components.py:190: UserWarning: `central_alignment_strength` not detected in the table, using value in self.param_dict.
  warn(msg)
/media/nvanalfen/T7/Northeastern/Research/my halotools/halotools/halotools/empirical_models/ia_models/ia_model_components.py:520: UserWarning: `satellite_alignment_strength` key not detected in `table`.The value set in self.param_dict of this class will be used instead.
  warn(msg)


As easy as that. If we want, we can now view the galaxy table

In [6]:
model_instance.mock.galaxy_table[:5]

halo_upid,halo_vy,_subhalo_inheritance_id,halo_mpeak,halo_y,halo_rvir,halo_axisA_y,halo_id,halo_hostid,halo_vx,halo_x,halo_mvir_host_halo_bin_number,halo_z,halo_axisA_z,halo_vz,halo_mvir_host_halo,halo_mvir,halo_axisA_x,halo_num_centrals,halo_num_satellites,gal_type,vy,galaxy_axisA_z,galaxy_axisC_x,galaxy_axisA_y,z,galaxy_axisC_y,x,real_subhalo,galaxy_axisA_x,vz,galaxy_axisB_x,galaxy_axisB_z,galaxy_axisC_z,galaxy_axisB_y,vx,y
int64,float32,int64,float64,float32,float32,float32,int64,int64,float32,float32,int64,float32,float32,float32,float32,float32,float32,int32,int32,object,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32
-1,142.24,689924,179099992064.0,47.75759,0.114595,5.49892,2812753737,2812753737,-313.51,59.33419,3,206.11255,-5.86109,28.29,179100000000.0,179100000000.0,-3.80638,1,0,centrals,142.24,-0.42755228,0.9862721,0.90388715,206.11255,0.08256559,59.33419,False,-0.013675861,28.29,-0.16456094,0.89260787,0.1430044,0.41972718,-313.51,47.75759
-1,332.93,733748,260000006144.0,188.50624,0.121388,2.75186,2816927677,2816927677,512.41,144.0893,3,155.66914,2.93946,171.44,212900000000.0,212900000000.0,6.45063,1,0,centrals,332.93,-0.44413498,0.37484723,0.17378844,155.66914,0.8274911,144.0893,False,-0.8789435,171.44,-0.29486918,0.7924621,-0.41802868,0.53390634,512.41,188.50624
-1,-19.71,734685,213799993344.0,17.493,0.121565,-2.10174,2810856805,2810856805,-10.74,43.4098,3,36.52288,7.61044,-308.07,213800000000.0,213800000000.0,0.67711,1,0,centrals,-19.71,-0.5077041,-0.46386406,0.52843094,36.52288,-0.8469444,43.4098,False,-0.68043905,-308.07,0.5673031,-0.8214142,-0.25983718,-0.058702294,-10.74,17.493
-1,253.17,754390,235699994624.0,188.20416,0.125185,6.13927,2820092494,2820092494,-128.43,200.56201,3,153.52306,-2.91432,219.06,233500000000.0,233500000000.0,-2.7247,1,0,centrals,253.17,0.31779182,-0.7513775,0.94769114,153.52306,-0.18841346,200.56201,False,0.029831069,219.06,-0.6591979,-0.70645326,0.6324019,0.25764686,-128.43,188.20416
-1,193.15,755442,236000002048.0,149.3881,0.125378,0.07564,2810092159,2810092159,382.7,22.71359,3,69.14876,6.94361,-148.26,234600000000.0,234600000000.0,-2.1641,1,0,centrals,193.15,0.20515724,-0.4350715,-0.65240085,69.14876,0.67420805,22.71359,False,-0.7295777,-148.26,0.5276639,0.77572817,0.59678835,-0.3461454,382.7,149.3881


As we see, there are now columns for galaxy_axisA_x (and y, and z). We have successfully incorporated the alignment model component into</br>
a halotools HOD model!

### Using a subhalo alignment model
While earlier, we used RadialSatellitesAlignment, here we will use SubhaloAlignment to align the satellites with respect to their own subhalos.</br>
As mentioned earlier, we will not be using the default subhalo inheritence dictionary. This means we will need to tell the HOD model to inherit</br>
certain extra parameters in the alignment model component.</br></br>

To do this, we <strong>MUST</strong> use SubhaloPhaseSpace as our satellite phase space model. Additionally, we must use the proper inherited_subhalo_props_dict</br>
for SubhaloAlignment. This dict is in subhalo_phase_space.py (the same file as SubhaloPhaseSpace) and is called alignment_inherited_subhalo_props_dict.

In [7]:
# Import the alignment_inherited_subhalo_props_dict
from halotools.empirical_models.phase_space_models.subhalo_based_models.subhalo_phase_space import alignment_inherited_subhalo_props_dict

With the proper inheritance dictionary, the following steps are <strong>ALMOST</strong> the exact same as before.</br></br>

The important difference being that when we instantiate our sats_prof_model using SubhaloPhaseSpace, in addition to the arguments</br>
passed in, we include the keyword argument "inherited_subhalo_props_dict" and set it equal to the dictionary we just imported</br></br>

Note also that in SubhaloAlignment, we must pass in the halocat. This lets the alignment model component retain access to the full halo catalog,</br>
which it uses to help reorient subhalos in the cases where other subhalos have been used in places where they do not truly exist.</br>
These instances will also show up in the galaxy_table as False in the "real_subhalo" column

In [8]:
# MODELS

# Alignment Strengths
satellite_alignment = 0.6
central_alignment = 0.8

# Central galaxy components
cens_occ_model = Zheng07Cens()
cens_prof_model = TrivialPhaseSpace()
cens_orientation_model = CentralAlignment(central_alignment_strength=central_alignment)

# Satellite galaxy Components
sats_occ_model = Zheng07Sats()
prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
sats_prof_model = SubhaloPhaseSpace(*prof_args, inherited_subhalo_props_dict=alignment_inherited_subhalo_props_dict)
sats_orientation_model = SubhaloAlignment(satellite_alignment_strength=satellite_alignment, halocat=halocat)

In [9]:
# Initially create the mock with SubhaloAlignment to overwrite the host information with subhalo
model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     centrals_orientation = cens_orientation_model,
                                     satellites_orientation = sats_orientation_model,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )

#seed=132358712
seed=None
model_instance.populate_mock(halocat, seed=seed)

/media/nvanalfen/T7/Northeastern/Research/my halotools/halotools/halotools/empirical_models/ia_models/ia_model_components.py:190: UserWarning: `central_alignment_strength` not detected in the table, using value in self.param_dict.
  warn(msg)
/media/nvanalfen/T7/Northeastern/Research/my halotools/halotools/halotools/empirical_models/ia_models/ia_model_components.py:879: UserWarning: `satellite_alignment_strength` not detected in the table, using value in self.param_dict.
  warn(msg)


Just as before, we get an HOD model instance with essentially the same construction.

In [10]:
model_instance.mock.galaxy_table[:5]

halo_upid,halo_vy,_subhalo_inheritance_id,halo_mpeak,halo_y,halo_rvir,halo_axisA_y,halo_id,halo_hostid,halo_vx,halo_x,halo_mvir_host_halo_bin_number,halo_z,halo_axisA_z,halo_vz,halo_mvir_host_halo,halo_mvir,halo_axisA_x,halo_num_centrals,halo_num_satellites,gal_type,subhalo_vy,vy,subhalo_rvir,galaxy_axisA_z,galaxy_axisC_x,subhalo_y,subhalo_vx,galaxy_axisA_y,z,subhalo_axisA_x,subhalo_x,galaxy_axisC_y,x,real_subhalo,galaxy_axisA_x,subhalo_vz,subhalo_upid,subhalo_axisA_z,subhalo_mvir,vz,subhalo_axisA_y,subhalo_z,galaxy_axisB_x,galaxy_axisB_z,galaxy_axisC_z,galaxy_axisB_y,vx,y
int64,float32,int64,float64,float32,float32,float32,int64,int64,float32,float32,int64,float32,float32,float32,float32,float32,float32,int32,int32,object,float64,float32,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,bool,float32,float64,int64,float64,float64,float32,float64,float64,float32,float32,float32,float32,float32,float32
-1,-146.49,694718,182199992320.0,33.9048,0.115251996,4.54161,2814302315,2814302315,-265.62,94.4852,3,125.78724,3.2406,-158.9,182200000000.0,182200000000.0,-4.4058,1,0,centrals,0.0,-146.49,0.0,0.8213909,-0.41559106,0.0,0.0,-0.041707985,125.78724,0.0,0.0,0.8764047,94.4852,False,-0.56883866,0.0,0,0.0,0.0,-158.9,0.0,0.0,0.70972294,0.51586634,-0.24330826,0.47976586,-265.62,33.9048
-1,-66.94,719106,200399994880.0,245.20497,0.118954,6.93008,2817092347,2817092347,292.54,148.39645,3,125.80485,2.18979,50.49,200400000000.0,200400000000.0,-3.1892,1,0,centrals,0.0,-66.94,0.0,-0.0839682,-0.35927087,0.0,0.0,0.6174169,125.80485,0.0,0.0,-0.33675537,148.39645,False,-0.7821417,0.0,0,0.0,0.0,50.49,0.0,0.0,-0.509096,-0.48521033,0.8703564,-0.71090937,292.54,245.20497
-1,-179.06,730923,210400002048.0,183.97427,0.120915,-7.41303,2810339199,2810339199,261.58,25.67056,3,107.1958,2.95114,-665.65,210400000000.0,210400000000.0,7.53031,1,0,centrals,0.0,-179.06,0.0,0.14933054,-0.4323325,0.0,0.0,-0.757522,107.1958,0.0,0.0,-0.5093699,25.67056,False,0.63550043,0.0,0,0.0,0.0,-665.65,0.0,0.0,-0.6397092,0.6512062,-0.7440638,-0.4082924,261.58,183.97427
-1,359.16,753467,232399994880.0,34.07706,0.124991,6.70509,2812706120,2812706120,167.08,66.22626,3,127.50671,5.19212,27.0,232400000000.0,232400000000.0,-0.44072,1,0,centrals,0.0,359.16,0.0,-0.66723937,-0.84194475,0.0,0.0,-0.6053868,127.50671,0.0,0.0,0.5360862,66.22626,False,-0.43393368,0.0,0,0.0,0.0,27.0,0.0,0.0,-0.32067204,0.7423281,0.061160523,-0.58831835,167.08,34.07706
-1,52.84,757879,237300006912.0,48.83463,0.125846,9.53717,2811177709,2811177709,-249.68,34.4197,3,181.64655,6.99473,270.32,237300000000.0,237300000000.0,10.1726,1,0,centrals,0.0,52.84,0.0,0.60852945,0.81275034,0.0,0.0,0.542402,181.64655,0.0,0.0,-0.44177243,34.4197,False,0.57921666,0.0,0,0.0,0.0,270.32,0.0,0.0,-0.06280873,0.6967194,-0.37983415,-0.71458876,-249.68,48.83463


## Using an IA strength model

The examples above show how to create an HOD model instance using halotools, but every central galaxy in those examples shares</br>
the same alignment strength as every other central galaxy, and every satellite galaxy shares the same alignment strength with every other</br>
satellite galaxy.</br></br>

To add flexibility, we have a fourth codel component called an IA Strength model. Essentially what this does is it takes halo and agalxy</br>
properties and converts them into an alignment strength. This way, every galaxy will have its own alignment strength based on its own</br>
parameters.</br></br>

Note, however, that the alignment process is probabilistic. The misalignment angle for each galaxy is drawn from a Dimroth-Watson distribution</br>
whose shape is determined by the alignment strength. THis means that two galaxies with the same alignment strength will nt necessarily</br>
have the same misalignment angle (they could be wildly different), and that a galaxy with a lower alignment strength could still align</br>
more closely to its reference vector than a galaxy with a higher alignment strength. A high alignment strength means the Dimroth-Watson</br>
will be a very narrow peak around perfect alignment. This would mean that a high alignment strength means that a galaxy is veery likely</br>
to align very closely on average.</br></br>

Here, we will look at the RadialSatellitesAlignmentStrength model, which assigns alignment strengths to satellites following a power law</br>
based on how close they are to their entral galaxy. At the time of writing this tutorial, this is the only IA strength model for</br>
satellite galaxies, but more are being planned and worked on.

In [11]:
# Import the RadialSAtellitesAlignmentStrength model
from halotools.empirical_models.ia_models.ia_strength_models import RadialSatelliteAlignmentStrength

We will return to using the RadialSatelliteAlignment from before (as it makes the most sense to vary with distance).</br>
Note that this means we will return to using the default subhalo_props_dict in subhaloPhaseSpace</br></br>

The RadialSatelliteAlignmentStrength component determines alignment strength via the equation $\mu = a*(\frac{r}{r_{vir}})^\gamma$</br>
where $a$ and $\gamma$ are out IA strength parameters, $r$ is the distance between the satellite galaxy and central galaxy,</br>
and $r_{vir}$ is the virial radius of the host halo.

In [12]:
# MODELS

# Alignment Strengths
satellite_alignment = 0.6
central_alignment = 0.8

# IA strength parameters
a = 0.8
gamma = -0.03

# Central galaxy components
cens_occ_model = Zheng07Cens()
cens_prof_model = TrivialPhaseSpace()
cens_orientation_model = CentralAlignment(central_alignment_strength=central_alignment)

# Satellite galaxy Components
sats_occ_model = Zheng07Sats()
prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
sats_prof_model = SubhaloPhaseSpace(*prof_args)
sats_orientation_model = RadialSatelliteAlignment(satellite_alignment_strength=satellite_alignment)

# Satellite strength model
sats_strength_model = RadialSatelliteAlignmentStrength(satellite_alignment_a=a, satellite_alignment_gamma=gamma)
sats_strength_model.inherit_halocat_properties(Lbox=halocat.Lbox)

Now create the model instance. Note again that the order of the model components is very important. The IA strength model must come before the orientation model that it is intended for. Here, as long as it comes before the satellites_orientation model, it's fine.

In [13]:
# Initially create the mock with SubhaloAlignment to overwrite the host information with subhalo
model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     satellites_radial_alignment_strength = sats_strength_model,
                                     centrals_orientation = cens_orientation_model,
                                     satellites_orientation = sats_orientation_model,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'satellites_radial_alignment_strength',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )

#seed=132358712
seed=None
model_instance.populate_mock(halocat, seed=seed)

/media/nvanalfen/T7/Northeastern/Research/my halotools/halotools/halotools/empirical_models/ia_models/ia_model_components.py:190: UserWarning: `central_alignment_strength` not detected in the table, using value in self.param_dict.
  warn(msg)


In [14]:
model_instance.mock.galaxy_table[:5]

halo_upid,halo_vy,_subhalo_inheritance_id,halo_mpeak,halo_y,halo_rvir,halo_axisA_y,halo_id,halo_hostid,halo_vx,halo_x,halo_mvir_host_halo_bin_number,halo_z,halo_axisA_z,halo_vz,halo_mvir_host_halo,halo_mvir,halo_axisA_x,halo_num_centrals,halo_num_satellites,gal_type,vy,galaxy_axisA_z,galaxy_axisC_x,galaxy_axisA_y,z,satellite_alignment_strength,galaxy_axisC_y,x,real_subhalo,galaxy_axisA_x,vz,galaxy_axisB_x,galaxy_axisB_z,galaxy_axisC_z,galaxy_axisB_y,vx,y
int64,float32,int64,float64,float32,float32,float32,int64,int64,float32,float32,int64,float32,float32,float32,float32,float32,float32,int32,int32,object,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32
-1,259.3,704428,189099999232.0,121.77341,0.116671994,0.91472,2811519774,2811519774,-2.51,38.25502,3,118.92752,6.62054,-238.03,189100000000.0,189100000000.0,2.41595,1,0,centrals,259.3,-0.34991255,0.4127311,-0.24286431,118.92752,0.0,0.29691273,38.25502,False,0.9047531,-238.03,0.105237335,-0.36887038,0.8611015,0.92350405,-2.51,121.77341
-1,403.02,711014,194999992320.0,129.05438,0.117683,-2.68788,2810144527,2810144527,-207.58,16.01311,3,133.66336,1.42741,22.94,194000000000.0,194000000000.0,6.76578,1,0,centrals,403.02,-0.2104285,0.357255,0.6276092,133.66336,0.0,0.11593043,16.01311,False,-0.7495509,22.94,0.5572632,0.31111228,-0.9267842,0.7698486,-207.58,129.05438
-1,-89.86,719040,227499999232.0,113.22632,0.118954,5.90154,2817804471,2817804471,-69.15,191.22415,3,25.03954,4.74046,-421.39,200400000000.0,200400000000.0,-1.73171,1,0,centrals,-89.86,0.95956695,0.10418287,0.20184815,25.03954,0.0,-0.96875405,191.22415,False,-0.19618507,-421.39,-0.9750166,-0.16902597,0.22508106,-0.14412798,-69.15,113.22632
-1,116.46,720632,201600008192.0,120.55582,0.119199,9.84778,2819451595,2819451595,-103.93,218.74976,3,108.82352,-4.1572,-82.19,201600000000.0,201600000000.0,-8.24602,1,0,centrals,116.46,0.36423543,0.8568757,0.8204941,108.82352,0.0,0.50535583,218.74976,False,-0.4405927,-82.19,0.2676604,0.9257176,-0.10187998,-0.26721692,-103.93,120.55582
-1,35.44,736682,215699996672.0,109.56529,0.121916,-3.91829,2816433172,2816433172,-5.34,153.79167,3,91.82954,4.78232,314.93,215700000000.0,215700000000.0,5.68385,1,0,centrals,35.44,0.57012194,-0.2833127,-0.19017313,91.82954,0.0,-0.95582235,153.79167,False,0.7992466,314.93,-0.53003657,0.8178162,0.07834286,0.22413807,-5.34,109.56529


Notice now the presence of the "satellite_alignmnet_strength" column in the galaxy table. This is what the IA strength model has done.</br>
When performing an alignment, the orientation model will first check to see if there is an appropriate alignment strength column in the</br>
table before using any parameters passed in as such. Of course, this value is 0.0 for all centrals since they would look for a</br>
"central_alignment_strength" column instead.